# Nonlinear Programming with Uncertainty

## Risk neutral objective: Maximize expected revenue

First, recall the deterministic waste disposal problem from Haith, D. A. (1982). *Environmental systems optimization*, solved in [this notebook](https://colab.research.google.com/github/EnvSystemsUVA/CodingExamples/blob/main/01_NLP_example1b.ipynb) using CPLEX with cvxpy.

\begin{align}
\text{Maximize} & \quad 0.4M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad 3M-2W(1-6\times10^{-6}W) \leq 100,000\\
& \quad M, W \geq 0
\end{align}

We saw in class that the stochastic problem doesn't change from the deterministic problem if we're trying to maximize the expected revenue and the value of the coefficient on M, $c$, is random with mean 0.4:

\begin{align}
\text{Maximize} & \quad \mathbb{E}[C]M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad 3M-2W(1-6\times10^{-6}W) \leq 100,000\\
& \quad M, W \geq 0
\end{align}

Below we reproduce this optimization.

In [ ]:
!pip install cplex

In [2]:
import cplex
import cvxpy as cvx

In [ ]:
limit = 100000
m = cvx.Variable()
w = cvx.Variable()
Ec = 0.4

obj = cvx.Maximize(Ec*m - 0.2*w)

constraints = [m <= 55000,
               w <= 70000,
               2*w <= 3*m,
               3*m - 2*w + 0.000012*w**2 <= limit]

prob = cvx.Problem(obj, constraints)
prob.solve(solver=cvx.CPLEX)
print('Objective (Expected revenue) = $%0.2f' % obj.value)
print('m = %0.2f kg metal/week' % m.value)
print('w = %0.2f m^3 wastewater/week' % w.value)

## Risk averse objective: Maximize 10th percentile revenue

Let's solve it for the risk-averse case where we want to maximize the 10th percentile revenue and $c\sim N(\mu=0.4,\sigma^2=0.2^2)$:

\begin{align}
\text{Maximize} & \quad c_{0.1}M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad 3M-2W(1-6\times10^{-6}W) \leq 100,000\\
& \quad M, W \geq 0\\
& \quad c_{0.1} = F_C^{-1}(0.1)\\
& \quad F_C(c) = \int_{-\infty}^c \frac{1}{\sigma\sqrt{2\pi}}\exp\Bigg(-\frac{(c-\mu)^2}{2\sigma^2}\Bigg)
\end{align}

First, let's see what the 10th percentile revenue would be from the above decision. To estimate that, we'll first need to find the 10th percentile of $c$.

In [ ]:
import scipy.stats as ss
c_01 = ss.norm.ppf(0.1,loc=0.4,scale=0.2)
print("10th percentile of c1: %0.2f" % c_01)

# scaling 10th percentile of standard normal for comparison
z_01 = ss.norm.ppf(0.1)
print("z_0.1: %0.2f" % z_01)
c_01 = 0.4 + 0.2*z_01
print("10th percentile of c1 check: %0.2f" % c_01)

print("10th percentile revenue: $%0.2f" % (c_01*m.value-0.2*w.value))

Now let's re-run the optimization with this risk averse formulation.

In [ ]:
obj = cvx.Maximize(c_01*m - 0.2*w)

constraints = [m <= 55000,
               w <= 70000,
               2*w <= 3*m,
               3*m - 2*w + 0.000012*w**2 <= limit,
               m >= 0,
               w >= 0]

prob = cvx.Problem(obj, constraints)
prob.solve(solver=cvx.CPLEX)
print('Objective (10th percentile revenue) = $%0.2f' % obj.value)
print('Average revenue = %0.2f' % (Ec*m.value-0.2*w.value))
print('m = %0.2f kg metal/week' % m.value)
print('w = %0.2f m^3 wastewater/week' % w.value)

If maximizing our 10th percentile revenue, we choose to produce 33,333.33 kg metal/week. This doesn't require any wastewater treatment to meet the effluent limit. It results in a 10th percentile revenue of \\$4,789.66, and an average revenue of \\$13,333.33.

When maximizing our average revenue, we chose to produce 45,486.11 kg metal/week and treat 20,833.34 m$^3$ of wastewater/week. This produced a higher average revenue of \$14,027.78, but a lower 10th percentile revenue of \$2,369.22.

## Another risk aversion objective: Maximize CVaR below 10th percentile

Now let's try to maximize the conditional value at risk (CVaR) below the 10th percentile, i.e. the average revenue below the 10th percentile:

\begin{align}
\text{Maximize} & \quad \mathbb{E}[C|C<c_{0.1}]M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad 3M-2W(1-6\times10^{-6}W) \leq 100,000\\
& \quad M, W \geq 0\\
\end{align}

To do this we need to find the expected value of $c$ below the 10th percentile: $\mathbb{E}[C|C<c_{0.1}]$.

In [ ]:
EC_01 = ss.norm(loc=0.4,scale=0.2).expect(lambda x: x, ub=c_01, conditional=True)

print("E[c|c<c_0.1]: %0.2f" % EC_01)

Now this becomes the coefficient on the metal production in the objective function of our optimization problem, as shown below.

In [ ]:
obj = cvx.Maximize(EC_01*m - 0.2*w)

constraints = [m <= 55000,
               w <= 70000,
               2*w <= 3*m,
               3*m - 2*w + 0.000012*w**2 <= limit,
               m >= 0,
               w >= 0]

prob = cvx.Problem(obj, constraints)
prob.solve(solver=cvx.CPLEX)
print('Objective (CVaR below 10th percentile) = $%0.2f' % obj.value)
print('10th percentile revenue = $%0.2f' % (c_01*m.value-0.2*w.value))
print('Average revenue = $%0.2f' % (Ec*m.value-0.2*w.value))
print('m = %0.2f kg metal/week' % m.value)
print('w = %0.2f m^3 wastewater/week' % w.value)

The optimal decision for this problem happens to be the same as the optimal decision when minimizing the 10th percentile itself, but this won't always be the case.

## Risk neutral constraint:

Now let's consider the risk neutral objective, but with the metal waste constraint reformulated as a concentration constraint $C$, which depends on uncertain flows, $Q$. First, we'll consider a risk neutral representation of the constraint in which we want our average concentration to be below $C$.

\begin{align}
\text{Maximize} & \quad \mathbb{E}[C]M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad 3M-2W(1-6\times10^{-6}W) \leq
\frac{C}{\mathbb{E}\{1/Q\}}\\
& \quad M, W \geq 0
\end{align}

To calculate $\mathbb{E}[1/Q]$, we need to know its distribution. In class we showed that if $X=1/Q$, and $Y=\ln(Q)\sim N(\mu_y,\sigma_y^2)$, the pdf of $X$ is:  

$f_X(x) = \frac{1}{\sigma_y x\sqrt{2\pi}} \exp\Big(-\frac{(\ln(1/x)-\mu_y)^2}{2\sigma_y^2}\Big)$

Below we write a class for this distribution with a method to calculate its PDF from the above formula as well as its CDF, inverse CDF (PPF), moments, and central moments from its PDF.

In [9]:
import scipy.integrate as integrate
from scipy.optimize import minimize
import numpy as np

class ConcentrationDistn:
    def __init__(self, mu_y, sigma_y):
        self.mu_y = mu_y # mean of Y = ln(Q)
        self.sigma_y = sigma_y # standard deviation of Y = ln(Q)

    def pdf(self, x):
        f_x = np.exp(-(np.log(1/x)-self.mu_y)**2 / (2*self.sigma_y**2) ) / (self.sigma_y * x * np.sqrt(2*np.pi))
        return f_x

    def cdf(self, x):
        # integrate pdf from ~0 to x
        return integrate.quad(lambda x: self.pdf(x), 0.000001, x)[0]

    def ppf(self, q, x0):
        # invert cdf at q by finding value of x for which absolute value of difference
        # between q and cdf(x) is minimized (it will be 0 at the inverse)
        f_inv_opt = lambda x: np.abs(X.cdf(x) - q)
        result = minimize(f_inv_opt, x0 = x0)
        return result.x[0]

    def moment(self, m):
        # integrate x^m * f(x) from ~0 to infinity
        return integrate.quad(lambda x: (x**m * self.pdf(x)), 0.000001, np.inf)[0]

    def central_moment(self, m):
        # integrate (x-mu)^m * f(x) from ~0 to infinity
        mu = self.moment(1)
        return integrate.quad(lambda x: ((x-mu)**m * self.pdf(x)), 0.000001, np.inf)[0]

Now let's create an object of this class and use it to calculate the $\mathbb{E}[X]$ if $\mu_y=9$, $\sigma_y=2$.

In [ ]:
mu_y = 9
sigma_y = 2
X = ConcentrationDistn(mu_y, sigma_y)
print("E[X] = E[1/Q] = %0.5f" % X.moment(1))

Now let's run the stochastic optimization with this risk neutral constraint assuming $C=100$ $kg/m^3$.

In [ ]:
C = 100

obj = cvx.Maximize(Ec*m - 0.2*w)

constraints = [m <= 55000,
               w <= 70000,
               2*w <= 3*m,
               3*m - 2*w + 0.000012*w**2 <= C / X.moment(1),
               m >= 0,
               w >= 0]

prob = cvx.Problem(obj, constraints)
prob.solve(solver=cvx.CPLEX)
print('Objective (expected revenue) = $%0.2f' % obj.value)
print('m = %0.2f kg metal/week' % m.value)
print('w = %0.2f m^3 wastewater/week' % w.value)

## Risk averse (chance) constraint:

Finally, we'll consider the risk neutral objective, but with the metal waste constraint reformulated as a *chance constraint* that the concentration $C$ is met with some probability $p$, here 0.9. This changes the optimization problem to the following:

\begin{align}
\text{Maximize} & \quad \mathbb{E}[C]M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad P(1/Q \leq C / [3M-2W(1-6\times10^{-6}W)] ) \geq 0.9\\
& \quad M, W \geq 0
\end{align}

With $X=1/Q$, this is equivalent to the following:

\begin{align}
\text{Maximize} & \quad \mathbb{E}[C]M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad F_X(C / [3M-2W(1-6\times10^{-6}W)]) \geq 0.9\\
& \quad M, W \geq 0
\end{align}

The chance constraint can be inverted so the decision variables are not within the CDF evaluation:

\begin{align}
\text{Maximize} & \quad \mathbb{E}[C]M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad F_X^{-1}(0.9) \leq C / [3M-2W(1-6\times10^{-6}W)]\\
& \quad M, W \geq 0
\end{align}

To solve this with cvxpy, we then need to move the decision variables out of the denominator so it does not violate Disciplined Convex Programming (DCP) rules:

\begin{align}
\text{Maximize} & \quad \mathbb{E}[C]M - 0.2W\\
\text{s.t.} & \quad M \leq 55,000\\
& \quad W \leq 70,000\\
& \quad 3M - 2W \geq 0\\
& \quad [3M-2W(1-6\times10^{-6}W)] F_X^{-1}(0.9) \leq C\\
& \quad M, W \geq 0
\end{align}

The 0.9 quantile of $X$ can be found with the ppf function:

In [ ]:
# Pass q to be estimated (0.9) and initial estimate of x_q (0.0016)
print("90th percentile of X: %0.5f" % X.ppf(0.9, 0.0016))

And now we can solve the risk averse problem:

In [ ]:
obj = cvx.Maximize(Ec*m - 0.2*w)

constraints = [m <= 55000,
               w <= 70000,
               2*w <= 3*m,
               X.ppf(0.9, 0.0016) * (3*m - 2*w + 0.000012*w**2) <= C,
               m >= 0,
               w >= 0]

prob = cvx.Problem(obj, constraints)
prob.solve(solver=cvx.CPLEX)
print('Objective (expected revenue) = $%0.2f' % obj.value)
print('m = %0.2f kg metal/week' % m.value)
print('w = %0.2f m^3 wastewater/week' % w.value)

Not surprisingly, we produce much less metal (31095.31 kg/week compared to 48,708.21 kg/week) and treat almost as much wastewater (20,832.92 m$^3$/week compared to 20,835.32 m$^3$/week) in order to meet this constraint 90\% of the time instead of just on average. This results in significantly less revenue (\$8,271.53 compared to \$15,316.22)